In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import glob
import numpy as np
from PIL import Image
from sklearn.metrics import pairwise_distances, roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
import sys
sys.path.append("../")
from inception_resnet_v1 import inference
import pandas as pd
import seaborn as sns

In [2]:
def prewhiten(x):
    """
    A helper function to whiten an image, or a batch of images.
    Args:
        x: An image or batch of images.
    """
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        print(x.ndim)
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
#     std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    """
    Normalizes an embedding to have unit length in the l2 metric.
    Args:
        x: A batch of numpy embeddings
    """
    output = x / np.sqrt(np.maximum(np.sum(np.square(x),
                                           axis=axis,
                                           keepdims=True),
                                    epsilon))
    return output

In [3]:
class MyModel:
    def __init__(self, model_path, inputs=None, sess=None):
        if inputs is None:
            self.model_inputs = tf.placeholder(tf.float32, shape=(None, 160, 160, 3))
        else:
            self.model_inputs = inputs
        
        vars_before = tf.global_variables()
        self.net, self.endpoints = inference(
            self.model_inputs, keep_probability=1.0, bottleneck_layer_size=512, phase_train=False)
        vars_after = tf.global_variables()
        
        model_name = dataset_to_model_name["vggface2"]
        saver = tf.train.Saver(list(set(vars_after) - set(vars_before)))
        
        if sess is None:
            self.sess = tf.Session()
        else:
            self.sess = sess
            
        saver.restore(self.sess, model_path)
    
    def predict(self, inputs, batch_size, layer_to_target):
        if layer_to_target is None:
            return self.sess.run(self.net, feed_dict={self.model_inputs: inputs})
        else:
            return self.sess.run(self.endpoints[layer_to_target], feed_dict={self.model_inputs: inputs})
    
    def __del__(self):
        self.sess.close()

dataset_to_model_name = {
    "casia-webface": "20180408-102900",
    "vggface2": "20180402-114759"
}

dataset_to_ckpt_number = {
    "casia-webface": "90",
    "vggface2": "275"
}

def build_model(dataset_name, inputs=None, sess=None):
    model_name = dataset_to_model_name[dataset_name]
    model = MyModel(
        os.path.join(
            "/home/ivan/facenet/models",
            model_name,
            "model-{model_name}.ckpt-{ckpt_num}".format(
                model_name=dataset_to_model_name[dataset_name],
                ckpt_num=dataset_to_ckpt_number[dataset_name]
            )),
        inputs, 
        sess
    )
    return model

In [14]:
class FacialVectorsDatabase:
    def __init__(self, folder, model_name, epsilon, attack_strategy, image_format, num_clean, include_decoys=True):
        self.associated_identities = []
        self.vectors = []
        self.associated_paths = []
        
        identities = os.listdir(folder)
        
        self.graph = tf.Graph()
        
        with self.graph.as_default():
            self.model = build_model(model_name)

            for person_name in tqdm(identities):
                attack_subfolder = os.path.join(folder, person_name, attack_strategy)

                protected = os.listdir(attack_subfolder)
                
                if include_decoys:
                    for indx, other_identity in enumerate(protected):
                        protected_folder = os.path.join(
                            attack_subfolder, 
                            other_identity, 
                            "epsilon_{eps}".format(eps=epsilon), 
                            image_format
                        )

                        self._add_folder_for_person(
                            protected_folder, 
                            person_name,
                            exclude_endings=None,
                            max_imgs=-1
                        )
                        
                # These were the images we used for this person for adversarial modification
                used_images = [
                    x.split("/")[-1] \
                    for indx, x in enumerate(self.associated_paths) \
                    if self.associated_identities[indx] == person_name
                ]
                
                clean_folder = os.path.join(
                    folder, person_name, "community_naive_mean", protected[0], "epsilon_0.0", "png")
                self._add_folder_for_person(
                    clean_folder, 
                    person_name,
                    exclude_endings=set(used_images),
                    max_imgs=num_clean
                )
        
                
    def _load_one_facenet(self, path, crop_box=None, resize_size=None, prewhiten_img=True):
        img = Image.open(path).convert("RGB")
        
        if crop_box:
            img = img.crop(crop_box)
            
        if resize_size:
            img = img.resize((resize_size, resize_size))
            
        img = (np.array(img)).astype(np.float32)
        
        if prewhiten_img:
            img = prewhiten(img)
            
        return img
        

    def _load_folder_for_facenet(self, folder, exclude_endings=None, max_imgs=-1):
        paths_list = glob.glob(os.path.join(folder, "*"))
        len_before = len(paths_list)
        if not (exclude_endings is None):
            paths_list = [x for x in paths_list if not (x.split("/")[-1] in exclude_endings)]
            
        if max_imgs > 0:
            paths_list = paths_list[:max_imgs]
        
        final_imgs = [
            self._load_one_facenet(
                img_path, 
                prewhiten_img=False, 
                resize_size=None, 
                crop_box=None) for img_path in paths_list
        ]

        final_imgs = np.array(final_imgs)
        final_imgs = prewhiten(np.array(final_imgs))
        return final_imgs, paths_list

    
    def _compute_embeddings(self, images):
        return self.model.predict(np.array(images), batch_size=len(images), layer_to_target=None)
    
    
    def _add_folder_for_person(self, folder, person_name, exclude_endings=None, max_imgs=-1):
        images, paths_list = self._load_folder_for_facenet(folder, exclude_endings, max_imgs=max_imgs)
        
        if images is None:
            return
        
        vectors = self._compute_embeddings(images)
        self.vectors.extend(vectors)
        self.associated_identities.extend([person_name for _ in range(len(vectors))])
        self.associated_paths.extend(paths_list)
    
    def nearest_neighbor_to_img_at_path(self, query_path):
        with self.graph.as_default():
            query_vector = self._compute_embeddings(
                np.expand_dims(self._load_one_facenet(query_path, resize_size=160), axis=0))
        distances = pairwise_distances(query_vector, self.vectors, metric="cosine")[0]
        min_dist_indx = np.argmin(distances)
        return self.associated_identities[min_dist_indx], self.associated_paths[min_dist_indx]
    
    def closest_k_to_img_at_path(self, query_path, k):
        with self.graph.as_default():
            query_vector = self._compute_embeddings(
                np.expand_dims(self._load_one_facenet(query_path, resize_size=160), axis=0))
        distances = pairwise_distances(query_vector, self.vectors, metric="cosine")[0]
        assert len(distances) == len(self.vectors)
        
        min_dist_indices = np.argsort(distances)[:k]
        top_k_identities = np.take(self.associated_identities, min_dist_indices, axis=0)
        top_k_paths = np.take(self.associated_paths, min_dist_indices, axis=0)

        return top_k_identities, top_k_paths


def full_path_to_info(path):
    split = path.split("/")
    if path.startswith("/data/vggface/test_perturbed_sampled"):
        # split of path breaks down like this:
        #     0 
        #     1 data
        #     2 vggface
        #     3 test_perturbed_sampled
        #     4 {protector}
        #     5 community_naive_mean
        #     6 {protected}
        #     7 epsilon_0.0
        #     8 png
        #     9 35.png
        return "-".join([split[4], split[6], split[7], split[9]])
    else:
        # split of path breaks down like this:
        #     0 
        #     1 data
        #     2 vggface
        #     3 test_query_antisampled
        #     4 {protector}
        #     5 image_name.jpeg
        return "-".join([split[4], split[5]])
    
    
def measure_local_recall(
    faces_database,
    image_directory="/data/vggface/test_query_antisampled",
    num_query=10,
    verbose=False,
    k=1
):
    discovery = []
    true = []
    identified_as = []
    
    paths_of_query = []
    paths_of_nearest = []

    for protector in os.listdir(image_directory):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            os.path.join(image_directory, protector, "*")
        ))

        
        for i in np.random.choice(len(query_photos_paths), num_query):
            chosen_path = query_photos_paths[i]
            
            if k == 1:
                top_identity, top_identity_path = faces_database.nearest_neighbor_to_img_at_path(chosen_path)

                paths_of_query.append(full_path_to_info(chosen_path))
                paths_of_nearest.append(full_path_to_info(top_identity_path))

                true.append(protector)
                identified_as.append(top_identity)

                if top_identity == protector:
                    discovery.append(1.0)
                else:
                    discovery.append(0.0)
            else:
                top_identities, top_paths = faces_database.closest_k_to_img_at_path(chosen_path, k)
                top_identities = np.array(top_identities)
                discovery.append(np.mean(top_identities == protector))
                if verbose:
                    print("Closest", k, "images to", chosen_path, "are", "\n".join(top_paths))
                    print()

    if verbose:
        if k == 1:
            for true_id, recognized_id, query, nearest in zip(true, identified_as, paths_of_query, paths_of_nearest):
                print("Face of {true_id} identitifed as {recognized_id}. Nearest neighbor to {query} was {nearest}".format(
                    true_id=true_id, recognized_id=recognized_id, query=query, nearest=nearest))
#         else:
#             print("verbose not available for k>1")

#     return sum(discovery)/len(discovery)
    return np.mean(discovery)

In [12]:
def get_recall_once(
    network_to_evaluate,
    epsilon,
    attack_name,
    num_clean,
    run,
    k=1,
    verbose=False
):
    faces_db = FacialVectorsDatabase(
        folder="/data/vggface/test_perturbed_sampled",
        model_name=network_to_evaluate,
        epsilon=epsilon,
        attack_strategy=attack_name,
        image_format="png", 
        num_clean=num_clean,
        include_decoys= epsilon != 0.0
    )
    recall_at_k1 = measure_local_recall(faces_db, num_query=10, verbose=verbose, k=k)

    faces_db.model.sess.close()
    
    return {
        "num_clean": num_clean, 
        "run": run,
        "recall": recall_at_k1,
        "epsilon": epsilon,
        "k": k
    }

def generate_results_csv(
    network_to_evaluate,
    attack_name,
    k=1
):
    results = []
    
    for run in range(10):
        # get a run with all clean ones and no decous
        res = get_recall_once(
            network_to_evaluate,
            0.0,
            attack_name,
            -1, # negative or 0 number includes all
            run,
            k
        )
        results.append(res)
    
    for epsilon in [0.1, 0.25, 0.5]:
        for num_clean in range(1, 10, 2):
            for run in range(10):
                res = get_recall_once(
                    network_to_evaluate,
                    epsilon,
                    attack_name,
                    num_clean,
                    run,
                    k
                )
                results.append(res)
                
    results_pd = pd.DataFrame(results)

    results_pd["decoys_vs_clean"] = 36 / results_pd["num_clean"]
    results_pd.loc[results_pd.decoys_vs_clean < 0.0, "decoys_vs_clean"] = 0.0
    
    results_pd.to_csv(
        "/home/ivan/pascal_adversarial_faces/results/recall_at_{k}_{attack_name}_{network_to_evaluate}.csv".format(
            k=k,
            network_to_evaluate=network_to_evaluate,
            attack_name=attack_name
        ))

In [ ]:
for attack_name in ["mean_casia-webface", "mean_vggface2", "mean_Conv2d_4b_3x3_vggface2", "ensemble_casia-webface"]:
    for k in [5, 10]:
        for network_to_evaluate in ["vggface2", "casia-webface"]:
            generate_results_csv(
                network_to_evaluate=network_to_evaluate,
                attack_name=attack_name,
                k=k
            )

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275


100%|██████████| 19/19 [00:03<00:00,  5.34it/s]


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275


100%|██████████| 19/19 [00:02<00:00,  6.34it/s]


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275


...

100%|██████████| 19/19 [00:07<00:00,  2.61it/s]


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180408-102900/model-20180408-102900.ckpt-90


100%|██████████| 19/19 [00:07<00:00,  2.66it/s]


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180408-102900/model-20180408-102900.ckpt-90


100%|██████████| 19/19 [00:07<00:00,  2.63it/s]


In [15]:
res = get_recall_once(
    "vggface2",
    0.5,
    "mean_vggface2",
    2, # negative or 0 number includes all
    1,
    5,
    verbose=True
)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275


100%|██████████| 19/19 [00:07<00:00,  2.38it/s]


Closest 5 images to /data/vggface/test_query_antisampled/n001781/0049_01.jpg are /data/vggface/test_perturbed_sampled/n005303/mean_vggface2/n001781/epsilon_0.5/png/22.png
/data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n005427/mean_vggface2/n001781/epsilon_0.5/png/27.png

Closest 5 images to /data/vggface/test_query_antisampled/n001781/0205_02.jpg are /data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n001781/epsilon_0.5/png/22.png
/data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n001781/epsilon_0.5/png/22.png
/data/vggface/test_perturbe

Closest 5 images to /data/vggface/test_query_antisampled/n009232/0008_01.jpg are /data/vggface/test_perturbed_sampled/n005427/mean_vggface2/n009232/epsilon_0.5/png/17.png
/data/vggface/test_perturbed_sampled/n000029/mean_vggface2/n009232/epsilon_0.5/png/25.png
/data/vggface/test_perturbed_sampled/n008613/mean_vggface2/n009232/epsilon_0.5/png/25.png
/data/vggface/test_perturbed_sampled/n009114/mean_vggface2/n009232/epsilon_0.5/png/25.png
/data/vggface/test_perturbed_sampled/n003215/mean_vggface2/n009232/epsilon_0.5/png/17.png

Closest 5 images to /data/vggface/test_query_antisampled/n009232/0209_01.jpg are /data/vggface/test_perturbed_sampled/n000958/mean_vggface2/n009232/epsilon_0.5/png/17.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n009232/epsilon_0.5/png/25.png
/data/vggface/test_perturbed_sampled/n002763/mean_vggface2/n009232/epsilon_0.5/png/25.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n009232/epsilon_0.5/png/17.png
/data/vggface/test_perturbe

Closest 5 images to /data/vggface/test_query_antisampled/n003356/0093_01.jpg are /data/vggface/test_perturbed_sampled/n003215/mean_vggface2/n003356/epsilon_0.5/png/16.png
/data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n003356/epsilon_0.5/png/16.png
/data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n003356/epsilon_0.5/png/3.png
/data/vggface/test_perturbed_sampled/n005427/mean_vggface2/n003356/epsilon_0.5/png/42.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n003356/epsilon_0.5/png/16.png

Closest 5 images to /data/vggface/test_query_antisampled/n003356/0678_01.jpg are /data/vggface/test_perturbed_sampled/n002647/mean_vggface2/n003356/epsilon_0.5/png/16.png
/data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n003356/epsilon_0.5/png/16.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n003356/epsilon_0.5/png/16.png
/data/vggface/test_perturbed_sampled/n005427/mean_vggface2/n003356/epsilon_0.5/png/42.png
/data/vggface/test_perturbed

...

Closest 5 images to /data/vggface/test_query_antisampled/n002763/0066_01.jpg are /data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n002763/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n002503/mean_vggface2/n002763/epsilon_0.5/png/10.png
/data/vggface/test_perturbed_sampled/n002647/mean_vggface2/n002763/epsilon_0.5/png/5.png
/data/vggface/test_perturbed_sampled/n005359/mean_vggface2/n002763/epsilon_0.5/png/10.png
/data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n002763/epsilon_0.5/png/5.png

Closest 5 images to /data/vggface/test_query_antisampled/n002763/0187_01.jpg are /data/vggface/test_perturbed_sampled/n002647/mean_vggface2/n002763/epsilon_0.5/png/5.png
/data/vggface/test_perturbed_sampled/n005359/mean_vggface2/n002763/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n002763/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002763/epsilon_0.5/png/5.png
/data/vggface/test_perturbed_sa

Closest 5 images to /data/vggface/test_query_antisampled/n002503/0205_01.jpg are /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n002503/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n000029/mean_vggface2/n002503/epsilon_0.5/png/49.png
/data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n002503/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n009288/mean_vggface2/n002503/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n002647/mean_vggface2/n002503/epsilon_0.5/png/30.png

Closest 5 images to /data/vggface/test_query_antisampled/n002503/0065_01.jpg are /data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002503/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n005359/mean_vggface2/n002503/epsilon_0.5/png/49.png
/data/vggface/test_perturbed_sampled/n008613/mean_vggface2/n002503/epsilon_0.5/png/10.png
/data/vggface/test_perturbed_sampled/n003215/mean_vggface2/n002503/epsilon_0.5/png/9.png
/data/vggface/test_perturbed

Closest 5 images to /data/vggface/test_query_antisampled/n005359/0393_03.jpg are /data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n002763/epsilon_0.5/png/10.png
/data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n002763/epsilon_0.5/png/30.png
/data/vggface/test_perturbed_sampled/n005359/mean_vggface2/n005427/epsilon_0.5/png/5.png
/data/vggface/test_perturbed_sampled/n007548/mean_vggface2/n005427/epsilon_0.5/png/5.png
/data/vggface/test_perturbed_sampled/n002647/mean_vggface2/n005427/epsilon_0.5/png/1.png

Closest 5 images to /data/vggface/test_query_antisampled/n005359/0277_01.jpg are /data/vggface/test_perturbed_sampled/n001683/mean_vggface2/n005359/epsilon_0.5/png/20.png
/data/vggface/test_perturbed_sampled/n005303/mean_vggface2/n005359/epsilon_0.5/png/31.png
/data/vggface/test_perturbed_sampled/n003215/mean_vggface2/n005359/epsilon_0.5/png/13.png
/data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n005359/epsilon_0.5/png/20.png
/data/vggface/test_perturbed_s

Closest 5 images to /data/vggface/test_query_antisampled/n007548/0112_04.jpg are /data/vggface/test_perturbed_sampled/n007548/community_naive_mean/n001781/epsilon_0.0/png/46.png
/data/vggface/test_perturbed_sampled/n009114/mean_vggface2/n007548/epsilon_0.5/png/44.png
/data/vggface/test_perturbed_sampled/n005303/mean_vggface2/n007548/epsilon_0.5/png/7.png
/data/vggface/test_perturbed_sampled/n000029/mean_vggface2/n007548/epsilon_0.5/png/21.png
/data/vggface/test_perturbed_sampled/n005303/mean_vggface2/n007548/epsilon_0.5/png/6.png

Closest 5 images to /data/vggface/test_query_antisampled/n007548/0172_03.jpg are /data/vggface/test_perturbed_sampled/n004658/mean_vggface2/n007548/epsilon_0.5/png/7.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n007548/epsilon_0.5/png/7.png
/data/vggface/test_perturbed_sampled/n002503/mean_vggface2/n007548/epsilon_0.5/png/7.png
/data/vggface/test_perturbed_sampled/n005303/mean_vggface2/n007548/epsilon_0.5/png/7.png
/data/vggface/test_perturb

Closest 5 images to /data/vggface/test_query_antisampled/n009114/0306_01.jpg are /data/vggface/test_perturbed_sampled/n009114/community_naive_mean/n001781/epsilon_0.0/png/39.png
/data/vggface/test_perturbed_sampled/n005359/mean_vggface2/n009288/epsilon_0.5/png/1.png
/data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n009288/epsilon_0.5/png/11.png
/data/vggface/test_perturbed_sampled/n000029/mean_vggface2/n009288/epsilon_0.5/png/1.png
/data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n009288/epsilon_0.5/png/11.png

Closest 5 images to /data/vggface/test_query_antisampled/n009114/0006_02.jpg are /data/vggface/test_perturbed_sampled/n002647/community_naive_mean/n001781/epsilon_0.0/png/46.png
/data/vggface/test_perturbed_sampled/n000029/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/test_perturbed_sampled/n004658/mean_vggface2/n005303/epsilon_0.5/png/31.png
/data/vggface/test_perturbed_sampled/n008613/mean_vggface2/n001781/epsilon_0.5/png/27.png
/data/vggface/t